In [3]:
import cv2
import numpy as np
import mediapipe as mp
import time
import os

In [6]:
# Main program entry point
if __name__ == "__main__":
    try:
        # Initialize camera
        cap = cv2.VideoCapture(0)
        
        # Check if camera is opened successfully
        if not cap.isOpened():
            print("Error: Could not open camera.")
            exit()
        
        # Create and initialize windows before the loop
        cv2.namedWindow("Camera", cv2.WINDOW_NORMAL)
        cv2.namedWindow("EyeType Interface", cv2.WINDOW_NORMAL)
        
        # Resize windows to reasonable dimensions
        cv2.resizeWindow("Camera", 640, 480)
        cv2.resizeWindow("EyeType Interface", 1000, 700)
        
        # Pre-initialize the interface to ensure it's visible
        menu_interface = np.zeros((700, 1000, 3), np.uint8)
        menu_interface[:] = COLORS['background']
        
        # Initialize cursor position
        cursor_pos = [500, 350]
        
        # First draw of the menu
        draw_menu()
        cv2.imshow("EyeType Interface", menu_interface)
        
        print("EyeType system starting...")
        print("Press ESC to exit or use the Exit button in the interface")
        
        while True:
            if exit_requested:
                print("Exit requested. Closing application...")
                break
                
            ret, frame = cap.read()
            if not ret:
                print("Error: Failed to capture image")
                break
                
            # Flip for mirror effect
            frame = cv2.flip(frame, 1)
            
            # Convert to RGB for MediaPipe
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = face_mesh.process(rgb_frame)
            
            if results.multi_face_landmarks:
                for face_landmarks in results.multi_face_landmarks:
                    # Get eye landmarks
                    left_eye = [(face_landmarks.landmark[i].x * frame.shape[1], face_landmarks.landmark[i].y * frame.shape[0]) 
                                for i in LEFT_EYE]
                    right_eye = [(face_landmarks.landmark[i].x * frame.shape[1], face_landmarks.landmark[i].y * frame.shape[0]) 
                                 for i in RIGHT_EYE]

                    # Calculate EAR
                    left_ear = calculate_ear(left_eye)
                    right_ear = calculate_ear(right_eye)
                    avg_ear = (left_ear + right_ear) / 2

                    # Draw eye landmarks
                    for point in left_eye + right_eye:
                        cv2.circle(frame, (int(point[0]), int(point[1])), 2, COLORS['accent'], -1, cv2.LINE_AA)
                        
                    # Simple gaze estimation using eye midpoints
                    left_mid_x = sum(p[0] for p in left_eye) / len(left_eye)
                    left_mid_y = sum(p[1] for p in left_eye) / len(left_eye)
                    right_mid_x = sum(p[0] for p in right_eye) / len(right_eye)
                    right_mid_y = sum(p[1] for p in right_eye) / len(right_eye)
                    
                    eye_mid_x = (left_mid_x + right_mid_x) / 2
                    eye_mid_y = (left_mid_y + right_mid_y) / 2
                    
                    # Scale from camera frame to interface dimensions
                    # Add sensitivity multipliers
                    sensitivity_x = 4.5  # Horizontal sensitivity
                    sensitivity_y = 4.5  # Vertical sensitivity

                    # Center point of the frame (neutral position)
                    center_x = frame.shape[1] / 2
                    center_y = frame.shape[0] / 2

                    # Calculate offset from center and apply sensitivity
                    offset_x = (eye_mid_x - center_x) * sensitivity_x
                    offset_y = (eye_mid_y - center_y) * sensitivity_y

                    # Apply offset to center of interface
                    target_x = int(menu_interface.shape[1] / 2 + offset_x)
                    target_y = int(menu_interface.shape[0] / 2 + offset_y)

                    # Ensure target stays within bounds
                    target_x = max(0, min(menu_interface.shape[1] - 1, target_x))
                    target_y = max(0, min(menu_interface.shape[0] - 1, target_y))
                    
                    # Update cursor position with smoothing
                    cursor_pos = update_cursor_position(cursor_pos, [target_x, target_y], cursor_smooth_factor)

                    # Blink detection logic with improved filtering and cooldown
                    if blink_cooldown > 0:
                        # Decrement cooldown counter
                        blink_cooldown -= 1
                        blink_counter = 0  # Reset counter during cooldown
                    elif avg_ear < blink_threshold:
                        # Only count consecutive frames below threshold
                        blink_counter += 1
                        # Visual feedback for blink detection
                        if blink_counter > 2:  # Only show visual feedback after a few frames to reduce flickering
                            cv2.putText(frame, "BLINK", (frame.shape[1] - 120, 40), font, 1, COLORS['accent'], 2, cv2.LINE_AA)
                    else:
                        if blink_counter >= blink_frames_required:
                            # Selection confirmed with a blink
                            # Set cooldown to prevent immediate new blink detection
                            blink_cooldown = 15
                            
                            # Play sound for feedback if available
                            if SOUND_AVAILABLE:
                                try:
                                    winsound.PlaySound("SystemExclamation", winsound.SND_ALIAS)
                                except:
                                    pass
                            
                            # Handle selection based on current mode and cursor position
                            # Check for exit first (available in all modes)
                            if gaze_element == "exit":
                                # Exit button selected
                                exit_requested = True
                                print("Exit requested via blink. Closing application...")
                                break
                                
                            # Check for reset (available in all modes)
                            elif gaze_element == "reset":
                                # Reset button selected - clear the typed text
                                typed_text = ""
                                print("Text reset.")
                            
                            elif current_mode == "main_menu":
                                if gaze_element == "option" and gaze_index is not None:
                                    # Selected a keyboard section
                                    current_mode = f"keyboard_{gaze_index}"
                                elif gaze_element == "save":
                                    # Save text button selected
                                    save_typed_text(typed_text)
                            
                            elif current_mode.startswith("keyboard_"):
                                section_index = int(current_mode.split("_")[1])
                                suggestions = get_word_suggestions(typed_text)
                                
                                if gaze_element == "suggestion" and gaze_index is not None:
                                    # Selected a word suggestion
                                    selected_word = suggestions[gaze_index]
                                    
                                    # Process the selected word
                                    words = typed_text.split()
                                    
                                    if not words or typed_text.endswith(" "):
                                        # If no words or last word complete, just add the suggestion
                                        typed_text += selected_word + " "
                                    else:
                                        # Replace the current word with the suggestion
                                        typed_text = " ".join(words[:-1])  # All words except last one
                                        if typed_text:
                                            typed_text += " "  # Add space if there were previous words
                                        typed_text += selected_word + " "  # Add selected suggestion
                                
                                elif gaze_element == "key" and gaze_index is not None:
                                    # Selected a keyboard key
                                    keys = keyboard_sections[section_index]
                                    selected_key = keys[gaze_index]
                                    if selected_key == "SPACE":
                                        typed_text += " "
                                    elif selected_key == "DEL":
                                        if typed_text:
                                            typed_text = typed_text[:-1]  # Remove last character
                                    else:
                                        typed_text += selected_key
                                
                                elif gaze_element == "section" and gaze_index is not None:
                                    # Switch directly to another keyboard section
                                    current_mode = f"keyboard_{gaze_index}"
                                    
                                elif gaze_element == "back":
                                    # Back to main menu
                                    current_mode = "main_menu"
                                
                                elif gaze_element == "save":
                                    # Save text button selected
                                    save_typed_text(typed_text)
                        
                        blink_counter = 0  # Reset blink counter
            
            # Draw the appropriate interface
            if current_mode == "main_menu":
                draw_menu()
            elif current_mode.startswith("keyboard_"):
                section_index = int(current_mode.split("_")[1])
                draw_keyboard_section(section_index)

            # Add subtle border to camera frame
            cv2.rectangle(frame, (0, 0), (frame.shape[1]-1, frame.shape[0]-1), (40, 40, 40), 1)

            # Show windows with modern naming
            cv2.imshow("Camera", frame)
            cv2.imshow("EyeType Interface", menu_interface)
            
            key = cv2.waitKey(1)
            if key == 27:  # Press ESC to exit
                break
            
            # Additional keyboard shortcuts for testing
            elif key == ord('s'):  # Press 's' to save text quickly
                save_typed_text(typed_text)
                # Show a confirmation message as toast notification
                overlay = menu_interface.copy()
                toast_width, toast_height = 300, 60
                toast_x = (menu_interface.shape[1] - toast_width) // 2
                toast_y = menu_interface.shape[0] - toast_height - 30
                
                # Draw toast background with rounded corners
                draw_rounded_rectangle(overlay, (toast_x, toast_y), 
                                      (toast_x + toast_width, toast_y + toast_height), 
                                      COLORS['success'], 8)
                
                # Add toast message
                cv2.putText(overlay, "Text Saved!", (toast_x + 70, toast_y + 40), 
                           font, 1, (255, 255, 255), 2)  # White text
                
                # Blend with original interface with fade effect
                cv2.addWeighted(overlay, 0.9, menu_interface, 0.1, 0, menu_interface)
                cv2.imshow("EyeType Interface", menu_interface)
                cv2.waitKey(500)  # Show message for 0.5 second
            
            elif key == ord('c') or key == ord('r'):  # Press 'c' or 'r' to clear/reset text
                typed_text = ""
                
            elif key == ord('m'):  # Press 'm' to return to main menu
                current_mode = "main_menu"
                
            elif key == ord('q') or key == ord('x'):  # Additional exit keys
                exit_requested = True
                break
        
        # Clean up
        print("Exiting EyeType system...")
        cap.release()
        cv2.destroyAllWindows()
        
    except Exception as e:
        print(f"An error occurred: {e}")
        
    finally:
        # Final cleanup to ensure resources are freed
        print("Cleaning up resources...")
        if 'cap' in locals() and cap is not None:
            cap.release()
        
        cv2.destroyAllWindows()
        cv2.waitKey(100)  # Give time for windows to close
        cv2.destroyAllWindows()  # Call again to ensure all windows closeimport cv2

try:
    import winsound
    SOUND_AVAILABLE = True
except ImportError:
    SOUND_AVAILABLE = False
    print("Winsound not available - sound feedback disabled")

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=False,
    max_num_faces=1,
    min_detection_confidence=0.5, 
    min_tracking_confidence=0.5
)

# Eye landmark indices (right and left eye)
LEFT_EYE = [33, 160, 158, 133, 153, 144]
RIGHT_EYE = [362, 385, 387, 263, 373, 380]

# Font for displaying text
font = cv2.FONT_HERSHEY_SIMPLEX


COLORS = {
    'background': (20, 20, 20),        
    'panel': (40, 40, 40),             
    'text': (255, 255, 255),           
    'accent': (0, 117, 255),           
    'success': (76, 187, 23),          
    'warning': (0, 149, 255),          
    'cursor': (0, 117, 255),           
    'key': (50, 50, 50),               
    'key_border': (70, 70, 70),        
    'key_text': (255, 255, 255),       
    'suggestion': (60, 60, 60),        
    'suggestion_border': (80, 80, 80), 
    'header': (15, 15, 15)             
}

# Initialize menu interface
menu_interface = np.zeros((700, 1000, 3), np.uint8)

# Define menu options
menu_options = {
    0: "Keyboard Section 1",
    1: "Keyboard Section 2",
    2: "Keyboard Section 3",
    3: "Keyboard Section 4"
}

# Define keyboard characters for each section
keyboard_sections = {
    0: ["A", "B", "C", "D", "E", "F", "G"],
    1: ["H", "I", "J", "K", "L", "M", "N"],
    2: ["O", "P", "Q", "R", "S", "T", "U"],
    3: ["V", "W", "X", "Y", "Z", "SPACE", "DEL"]
}

# Dictionary of common words for suggestions
common_words = {
    "a": ["and", "at", "about", "again", "all"],
    "b": ["but", "by", "be", "been", "because"],
    "c": ["can", "could", "come", "called", "create"],
    "d": ["do", "did", "down", "day", "don't"],
    "e": ["every", "eye", "even", "end", "ever"],
    "f": ["for", "from", "find", "first", "few"],
    "g": ["good", "get", "go", "great", "give"],
    "h": ["have", "had", "how", "help", "here"],
    "i": ["in", "it", "is", "if", "into"],
    "j": ["just", "job", "join", "jump", "joy"],
    "k": ["know", "key", "keep", "kind", "knowledge"],
    "l": ["like", "look", "long", "last", "live"],
    "m": ["more", "make", "many", "my", "most"],
    "n": ["not", "no", "now", "new", "next"],
    "o": ["of", "on", "or", "out", "one"],
    "p": ["people", "part", "place", "point", "provide"],
    "q": ["question", "quick", "quite", "quality", "quiet"],
    "r": ["right", "run", "really", "read", "result"],
    "s": ["so", "see", "some", "say", "should"],
    "t": ["the", "to", "time", "they", "than"],
    "u": ["up", "use", "us", "under", "until"],
    "v": ["very", "view", "value", "visit", "voice"],
    "w": ["with", "when", "will", "were", "what"],
    "x": ["example", "extra", "x-ray", "expect", "exchange"],
    "y": ["you", "your", "year", "yes", "yet"],
    "z": ["zero", "zone", "zoom", "zeal", "zebra"]
}

# Variables for tracking
typed_text = ""  # Store the typed text
blink_counter = 0
blink_threshold = 0.19  # EAR threshold for blink detection (slightly more strict)
blink_frames_required = 6  # Increased frames required to confirm a blink
blink_cooldown = 0  # Cooldown counter to prevent accidental double blinks
current_mode = "main_menu"  # Can be "main_menu" or "keyboard_X"
suggestion_mode = False
exit_requested = False  # Flag to track if exit was requested

# Cursor variables
cursor_pos = [500, 350]  # Initial cursor position
cursor_smooth_factor = 0.3  # Smoothing factor for cursor movement (0-1)
gaze_element = None  # Current element being gazed at
gaze_index = None  # Index of the current element

# Function to calculate Eye Aspect Ratio (EAR)
def calculate_ear(eye_landmarks):
    vertical_1 = np.linalg.norm(np.array(eye_landmarks[1]) - np.array(eye_landmarks[5]))
    vertical_2 = np.linalg.norm(np.array(eye_landmarks[2]) - np.array(eye_landmarks[4]))
    horizontal = np.linalg.norm(np.array(eye_landmarks[0]) - np.array(eye_landmarks[3]))
    ear = (vertical_1 + vertical_2) / (2.0 * horizontal)
    return ear

# Function to get word suggestions based on current word being typed
def get_word_suggestions(text):
    if not text:
        return ["I", "and", "to"]
    
    # Split text into words
    words = text.split()
    
    # If there's no word being typed, return common starters
    if not words or words[-1] == "":
        return ["I", "and", "to"]
    
    current_word = words[-1].lower()
    
    # If word is complete (followed by a space), suggest common next words
    if text.endswith(" "):
        return ["the", "and", "to"]
    
    # Get first letter of current word for dictionary lookup
    first_letter = current_word[0]
    
    if first_letter in common_words:
        # Filter suggestions that start with the current word
        suggestions = [word for word in common_words[first_letter] if word.startswith(current_word)]
        
        # If no matching suggestions, return the default for this letter
        if not suggestions:
            return common_words[first_letter][:3]
        
        # Pad with default suggestions if less than 3 matches
        while len(suggestions) < 3:
            for word in common_words[first_letter]:
                if word not in suggestions:
                    suggestions.append(word)
                    if len(suggestions) >= 3:
                        break
        
        return suggestions[:3]
    else:
        # If first letter not in dictionary, return general common words
        return ["the", "and", "to"]

# Function to save typed text to a file
def save_typed_text(text, filename="typed_text.txt"):
    with open(filename, "w") as file:
        file.write(text)
    print(f"Text saved to {filename}")

# Function to draw cursor
def draw_cursor(interface, x, y):
    # Outer glow effect
    cv2.circle(interface, (x, y), 8, COLORS['cursor'], -1, cv2.LINE_AA)
    # Inner brighter circle
    cv2.circle(interface, (x, y), 3, (255, 255, 255), -1, cv2.LINE_AA)

# Function to create a simple rounded rectangle
def draw_rounded_rectangle(img, top_left, bottom_right, color, radius=6, thickness=-1):
    x1, y1 = top_left
    x2, y2 = bottom_right
    
    # Draw main rectangle
    cv2.rectangle(img, (x1+radius, y1), (x2-radius, y2), color, thickness)
    cv2.rectangle(img, (x1, y1+radius), (x2, y2-radius), color, thickness)
    
    # Draw four corners
    cv2.ellipse(img, (x1+radius, y1+radius), (radius, radius), 180, 0, 90, color, thickness)
    cv2.ellipse(img, (x2-radius, y1+radius), (radius, radius), 270, 0, 90, color, thickness)
    cv2.ellipse(img, (x1+radius, y2-radius), (radius, radius), 90, 0, 90, color, thickness)
    cv2.ellipse(img, (x2-radius, y2-radius), (radius, radius), 0, 0, 90, color, thickness)

# Function to smoothly update cursor position based on gaze
def update_cursor_position(current_pos, target_pos, smoothing_factor):
    # Apply smoothing to reduce jitter
    new_x = int(current_pos[0] + (target_pos[0] - current_pos[0]) * smoothing_factor)
    new_y = int(current_pos[1] + (target_pos[1] - current_pos[1]) * smoothing_factor)
    return [new_x, new_y]

# Function to check if point is inside rectangle
def is_point_in_rect(point, rect_start, rect_end):
    x, y = point
    x1, y1 = rect_start
    x2, y2 = rect_end
    return x1 <= x <= x2 and y1 <= y <= y2

# Function to draw the main menu interface
def draw_menu():
    global gaze_element, gaze_index, menu_interface
    
    # Clear the interface with dark background
    menu_interface[:] = COLORS['background']
    
    # Header bar 
    cv2.rectangle(menu_interface, (0, 0), (1000, 60), COLORS['header'], -1)
    
    # Title
    cv2.putText(menu_interface, "EyeType", (40, 40), font, 1.2, COLORS['text'], 2)
    
    # Add home, reset and exit buttons to match screenshot
    # Reset button
    reset_btn_x, reset_btn_y = 550, 40
    
    # Check if cursor is over reset button
    if is_point_in_rect(cursor_pos, (reset_btn_x - 40, reset_btn_y - 20), (reset_btn_x + 70, reset_btn_y + 10)):
        gaze_element = "reset"
        cv2.putText(menu_interface, "Reset", (reset_btn_x, reset_btn_y), font, 0.7, COLORS['accent'], 1)
    else:
        cv2.putText(menu_interface, "Reset", (reset_btn_x, reset_btn_y), font, 0.7, COLORS['text'], 1)
    
    # Home button
    home_btn_x, home_btn_y = 650, 40
    cv2.putText(menu_interface, "Home", (home_btn_x, home_btn_y), font, 0.7, COLORS['text'], 1)
    
    # Exit button
    exit_btn_x, exit_btn_y = 750, 40
    
    # Check if cursor is over exit button
    if is_point_in_rect(cursor_pos, (exit_btn_x - 40, exit_btn_y - 20), (exit_btn_x + 70, exit_btn_y + 10)):
        gaze_element = "exit"
        cv2.putText(menu_interface, "Exit", (exit_btn_x, exit_btn_y), font, 0.7, COLORS['warning'], 1)
    else:
        cv2.putText(menu_interface, "Exit", (exit_btn_x, exit_btn_y), font, 0.7, COLORS['text'], 1)
    
    # Typed Text label
    cv2.putText(menu_interface, "Typed Text:", (40, 90), font, 0.7, COLORS['text'], 1)
    
    # Status area with typed text (dark panel)
    cv2.rectangle(menu_interface, (40, 100), (960, 160), COLORS['panel'], -1)
    cv2.rectangle(menu_interface, (40, 100), (960, 160), (60, 60, 60), 1)  # subtle border
    
    # Add text indicator
    if typed_text:
        cv2.putText(menu_interface, typed_text, (60, 140), font, 0.9, COLORS['text'], 2)
    
    # Reset gaze detection for other elements (keep exit/reset if active)
    if gaze_element not in ["exit", "reset"]:
        gaze_element = None
        gaze_index = None
    
    # Add "Keyboard Sections" panel on the left (no "How to use" section)
    panel_x, panel_y = 40, 200
    panel_w, panel_h = 200, 180  # Shorter panel, only for keyboard sections
    cv2.rectangle(menu_interface, (panel_x, panel_y), 
                 (panel_x + panel_w, panel_y + panel_h), 
                 COLORS['panel'], -1)
    cv2.rectangle(menu_interface, (panel_x, panel_y), 
                 (panel_x + panel_w, panel_y + panel_h), 
                 (60, 60, 60), 1)  # subtle border
    
    # Add keyboard section info title - using accent color
    cv2.putText(menu_interface, "Keyboard Sections:", (panel_x + 15, panel_y + 35), 
               font, 0.7, COLORS['accent'], 1)
    
    section_info = [
        "1: A-G",
        "2: H-N",
        "3: O-U",
        "4: V-Z, "
        "SPACE, DEL"
    ]
    
    # Draw section info with more spacing and larger font
    for i, info in enumerate(section_info):
        y_pos = panel_y + 70 + i*30  # Increased spacing between lines
        cv2.putText(menu_interface, info, (panel_x + 20, y_pos), 
                   font, 0.7, COLORS['text'], 1)  # Larger font size
    
    # Blink progress bar (minimal, non-overlapping design)
    if blink_counter > 0:
        # Position at top of menu, below header
        progress_bar_x = 780
        progress_bar_y = 80
        progress_bar_w = 200
        progress_bar_h = 10
        
        # Draw background track
        cv2.rectangle(menu_interface, 
                     (progress_bar_x, progress_bar_y), 
                     (progress_bar_x + progress_bar_w, progress_bar_y + progress_bar_h), 
                     (40, 40, 40), -1, cv2.LINE_AA)
        
        # Draw progress
        progress = int((blink_counter / blink_frames_required) * progress_bar_w)
        cv2.rectangle(menu_interface, 
                     (progress_bar_x, progress_bar_y), 
                     (progress_bar_x + progress, progress_bar_y + progress_bar_h), 
                     COLORS['accent'], -1, cv2.LINE_AA)
        
        # Add label
        cv2.putText(menu_interface, "Blink", (progress_bar_x - 40, progress_bar_y + 9), 
                   font, 0.5, COLORS['text'], 1)
    
    # Draw menu options in a grid with dark theme style
    option_width = 300
    option_height = 140
    
    # Add spacing between the left panel and keyboard sections (using the original spacing)
    gap_between_sections = 60
    
    positions = [
        (panel_x + panel_w + gap_between_sections, 200),                            # Top left
        (panel_x + panel_w + gap_between_sections + option_width + 40, 200),        # Top right
        (panel_x + panel_w + gap_between_sections, 200 + option_height + 50),       # Bottom left
        (panel_x + panel_w + gap_between_sections + option_width + 40, 200 + option_height + 50)  # Bottom right
    ]
    
    # Check all menu options
    for i in range(len(menu_options)):
        x, y = positions[i]
        
        # Check if cursor is over this option
        hovered = is_point_in_rect(cursor_pos, (x, y), (x + option_width, y + option_height))
        
        if hovered:
            gaze_element = "option"
            gaze_index = i
            # Highlighted option
            draw_rounded_rectangle(menu_interface, (x, y), (x + option_width, y + option_height), 
                        COLORS['accent'], 6)
            font_color = (255, 255, 255)  # White text
        else:
            # Non-highlighted options
            draw_rounded_rectangle(menu_interface, (x, y), (x + option_width, y + option_height), 
                        COLORS['panel'], 6)
            # Add subtle border
            cv2.rectangle(menu_interface, (x, y), (x + option_width, y + option_height), 
                       (60, 60, 60), 1)
            font_color = COLORS['text']
        
        # Draw text
        text = menu_options[i]
        text_size = cv2.getTextSize(text, font, 0.9, 2)[0]
        text_x = x + int((option_width - text_size[0]) / 2)
        text_y = y + int((option_height + text_size[1]) / 2)
        cv2.putText(menu_interface, text, (text_x, text_y), font, 0.9, font_color, 2)
    
    # Save text button - positioned at the bottom with proper spacing
    save_btn_x, save_btn_y = 400, 580
    save_btn_w, save_btn_h = 200, 50
    
    # Check if cursor is over save button
    if is_point_in_rect(cursor_pos, (save_btn_x, save_btn_y), (save_btn_x + save_btn_w, save_btn_y + save_btn_h)):
        gaze_element = "save"
        # Highlighted button
        draw_rounded_rectangle(menu_interface, (save_btn_x, save_btn_y), 
                    (save_btn_x + save_btn_w, save_btn_y + save_btn_h), 
                    COLORS['success'], 6)
        font_color = (255, 255, 255)  # White text
    else:
        draw_rounded_rectangle(menu_interface, (save_btn_x, save_btn_y), 
                    (save_btn_x + save_btn_w, save_btn_y + save_btn_h), 
                    COLORS['panel'], 6)
        # Add subtle border
        cv2.rectangle(menu_interface, (save_btn_x, save_btn_y), 
                     (save_btn_x + save_btn_w, save_btn_y + save_btn_h), 
                     (60, 60, 60), 1)
        font_color = COLORS['text']
    
    # Save text
    cv2.putText(menu_interface, "SAVE TEXT", (save_btn_x + 45, save_btn_y + 35), font, 0.8, font_color, 2)
    
    # Draw cursor
    draw_cursor(menu_interface, cursor_pos[0], cursor_pos[1])

# Function to draw the keyboard section
def draw_keyboard_section(section_index):
    global gaze_element, gaze_index, menu_interface
    
    # Clear the interface with dark background
    menu_interface[:] = COLORS['background']
    
    # Header bar
    cv2.rectangle(menu_interface, (0, 0), (1000, 60), COLORS['header'], -1)
    
    # Title with section indicator
    title = "EyeType"
    cv2.putText(menu_interface, title, (40, 40), font, 1.2, COLORS['text'], 2)
    
    # Add reset, home, and exit buttons
    # Reset button
    reset_btn_x, reset_btn_y = 550, 40
    
    # Check if cursor is over reset button
    if is_point_in_rect(cursor_pos, (reset_btn_x - 40, reset_btn_y - 20), (reset_btn_x + 70, reset_btn_y + 10)):
        gaze_element = "reset"
        cv2.putText(menu_interface, "Reset", (reset_btn_x, reset_btn_y), font, 0.7, COLORS['accent'], 1)
    else:
        cv2.putText(menu_interface, "Reset", (reset_btn_x, reset_btn_y), font, 0.7, COLORS['text'], 1)
    
    # Home button
    home_btn_x, home_btn_y = 650, 40
    cv2.putText(menu_interface, "Home", (home_btn_x, home_btn_y), font, 0.7, COLORS['text'], 1)
    
    # Exit button
    exit_btn_x, exit_btn_y = 750, 40
    
    # Check if cursor is over exit button
    if is_point_in_rect(cursor_pos, (exit_btn_x - 40, exit_btn_y - 20), (exit_btn_x + 70, exit_btn_y + 10)):
        gaze_element = "exit"
        cv2.putText(menu_interface, "Exit", (exit_btn_x, exit_btn_y), font, 0.7, COLORS['warning'], 1)
    else:
        cv2.putText(menu_interface, "Exit", (exit_btn_x, exit_btn_y), font, 0.7, COLORS['text'], 1)
    
    # Typed Text label
    cv2.putText(menu_interface, "Typed Text:", (40, 90), font, 0.7, COLORS['text'], 1)
    
    # Text area with typed text (dark panel)
    cv2.rectangle(menu_interface, (40, 100), (960, 160), COLORS['panel'], -1)
    cv2.rectangle(menu_interface, (40, 100), (960, 160), (60, 60, 60), 1)  # subtle border
    
    # Add text indicator
    if typed_text:
        cv2.putText(menu_interface, typed_text, (60, 140), font, 0.9, COLORS['text'], 2)
    
    # Reset gaze detection for other elements (keep exit/reset if active)
    if gaze_element not in ["exit", "reset"]:
        gaze_element = None
        gaze_index = None
    
    # Section title - make it larger and centered
    section_title = f"Keyboard Section {section_index+1}"
    title_size = cv2.getTextSize(section_title, font, 1.3, 2)[0]
    title_x = (menu_interface.shape[1] - title_size[0]) // 2
    cv2.putText(menu_interface, section_title, (title_x, 190), font, 1.3, COLORS['text'], 2)
    
    # Add keyboard sections info panel on the left (no "How to use" section)
    panel_x, panel_y = 40, 180
    panel_w, panel_h = 200, 180  # Shorter panel, only for keyboard sections
    cv2.rectangle(menu_interface, (panel_x, panel_y), 
                 (panel_x + panel_w, panel_y + panel_h), 
                 COLORS['panel'], -1)
    cv2.rectangle(menu_interface, (panel_x, panel_y), 
                 (panel_x + panel_w, panel_y + panel_h), 
                 (60, 60, 60), 1)  # subtle border
    
    # Add keyboard section info title - using accent color
    cv2.putText(menu_interface, "Keyboard Sections:", (panel_x + 15, panel_y + 35), 
               font, 0.7, COLORS['accent'], 1)
    
    section_info = [
        "1: A-G",
        "2: H-N",
        "3: O-U",
        "4: V-Z, SPACE, DEL"
    ]
    
    # Draw section info with more spacing and larger font
    for i, info in enumerate(section_info):
        # Highlight current section
        color = COLORS['accent'] if i == section_index else COLORS['text']
        y_pos = panel_y + 70 + i*30  # Increased spacing between lines
        cv2.putText(menu_interface, info, (panel_x + 20, y_pos), 
                   font, 0.7, color, 1)  # Larger font size
    
    # Blink progress bar (non-overlapping)
    if blink_counter > 0:
        # Position at top right, below header
        progress_bar_x = 780
        progress_bar_y = 80
        progress_bar_w = 200
        progress_bar_h = 10
        
        # Draw background track
        cv2.rectangle(menu_interface, 
                     (progress_bar_x, progress_bar_y), 
                     (progress_bar_x + progress_bar_w, progress_bar_y + progress_bar_h), 
                     (40, 40, 40), -1, cv2.LINE_AA)
        
        # Draw progress
        progress = int((blink_counter / blink_frames_required) * progress_bar_w)
        cv2.rectangle(menu_interface, 
                     (progress_bar_x, progress_bar_y), 
                     (progress_bar_x + progress, progress_bar_y + progress_bar_h), 
                     COLORS['accent'], -1, cv2.LINE_AA)
        
        # Add label
        cv2.putText(menu_interface, "Blink", (progress_bar_x - 40, progress_bar_y + 9), 
                   font, 0.5, COLORS['text'], 1)
    
    # Word suggestions area - aligned with the center and more spaced out
    suggestions = get_word_suggestions(typed_text)
    
    # For a cleaner look, draw only 3 word suggestions centered with better spacing
    suggestion_width = 160
    suggestion_height = 40
    suggestion_gap = 40  # Increased gap between suggestions
    suggestion_y = 240  # Moved down to provide more space from the title
    
    # ADD SPACING: Increase spacing between left panel and content
    gap_between_sections = 60  # Increased gap between the sections panel and keyboard content
    
    # Adjust the starting position to include the gap
    content_start_x = panel_x + panel_w + gap_between_sections
    
    # Calculate the available width for content
    available_width = menu_interface.shape[1] - content_start_x - 40  # 40px margin on right
    
    total_suggestions_width = (suggestion_width * 3) + (suggestion_gap * 2)
    start_x = content_start_x + (available_width - total_suggestions_width) // 2
    
    # Draw word suggestions
    for i, word in enumerate(suggestions[:3]):  # Limit to 3 suggestions
        x = start_x + i * (suggestion_width + suggestion_gap)
        
        # Check if cursor is over this suggestion
        if is_point_in_rect(cursor_pos, (x, suggestion_y), (x + suggestion_width, suggestion_y + suggestion_height)):
            gaze_element = "suggestion"
            gaze_index = i
            # Highlighted suggestion
            draw_rounded_rectangle(menu_interface, (x, suggestion_y), 
                                 (x + suggestion_width, suggestion_y + suggestion_height), 
                                 COLORS['accent'], 6)
            font_color = (255, 255, 255)  # White text
        else:
            # Normal suggestion
            draw_rounded_rectangle(menu_interface, (x, suggestion_y), 
                                 (x + suggestion_width, suggestion_y + suggestion_height), 
                                 COLORS['panel'], 6)
            # Add subtle border
            cv2.rectangle(menu_interface, (x, suggestion_y), 
                        (x + suggestion_width, suggestion_y + suggestion_height), 
                        (60, 60, 60), 1)
            font_color = COLORS['text']
        
        # Draw suggestion text with better font weight
        text_size = cv2.getTextSize(word, font, 0.9, 2)[0]  # Increased font size
        text_x = x + int((suggestion_width - text_size[0]) / 2)
        text_y = suggestion_y + int((suggestion_height + text_size[1]) / 2)
        cv2.putText(menu_interface, word, (text_x, text_y), font, 0.9, font_color, 2)
    
    # Draw keyboard keys (dark theme style) - improved spacing and larger fonts
    key_width = 80  # Larger key size
    key_height = 80  # Square keys
    key_gap = 30    # More space between keys
    
    # Calculate total width of keys in a row
    keys_per_row = 5
    total_row_width = (key_width * min(len(keyboard_sections[section_index]), keys_per_row)) + (key_gap * (min(len(keyboard_sections[section_index]), keys_per_row) - 1))
    
    # Center the keyboard in the content area (with the gap from the left panel)
    keyboard_start_x = content_start_x + (available_width - total_row_width) // 2
    keyboard_start_y = 320  # Position keyboard further below suggestions
    
    keys = keyboard_sections[section_index]
    for i, key in enumerate(keys):
        # Calculate position (5 keys per row max)
        row = i // keys_per_row
        col = i % keys_per_row
        
        x = keyboard_start_x + col * (key_width + key_gap)
        y = keyboard_start_y + row * (key_height + key_gap)
        
        # Check if cursor is over this key
        if is_point_in_rect(cursor_pos, (x, y), (x + key_width, y + key_height)):
            gaze_element = "key"
            gaze_index = i
            # Highlighted key
            draw_rounded_rectangle(menu_interface, (x, y), (x + key_width, y + key_height), 
                        COLORS['accent'], 6)
            font_color = (255, 255, 255)  # White text
        else:
            # Normal key
            draw_rounded_rectangle(menu_interface, (x, y), (x + key_width, y + key_height), 
                        COLORS['key'], 6)
            # Add subtle border
            cv2.rectangle(menu_interface, (x, y), (x + key_width, y + key_height), 
                       (70, 70, 70), 1)
            font_color = COLORS['key_text']
        
        # Draw key text with better font
        text_size = cv2.getTextSize(key, font, 1.1, 2)[0]  # Larger font
        text_x = x + int((key_width - text_size[0]) / 2)
        text_y = y + int((key_height + text_size[1]) / 2)
        cv2.putText(menu_interface, key, (text_x, text_y), font, 1.1, font_color, 2)
    
    # Section navigation buttons - properly spaced at the bottom
    nav_btn_width = 110
    nav_btn_height = 45
    nav_btn_gap = 20
    
    # Calculate total width of all section buttons
    all_sections_width = (nav_btn_width * 4) + (nav_btn_gap * 3)
    section_start_x = (menu_interface.shape[1] - all_sections_width) // 2
    section_y = 520  # Position above the back/save buttons with more space
    
    # Draw section navigation buttons
    for i in range(len(menu_options)):
        # Position buttons in a row
        nav_btn_x = section_start_x + (i * (nav_btn_width + nav_btn_gap))
        
        # Highlight current section differently
        if i == section_index:
            # Current section indicated with an accent color
            draw_rounded_rectangle(menu_interface, (nav_btn_x, section_y), 
                                 (nav_btn_x + nav_btn_width, section_y + nav_btn_height), 
                        COLORS['warning'], 6)
            font_color = (255, 255, 255)  # White text
        # Check if cursor is over this section button
        elif is_point_in_rect(cursor_pos, (nav_btn_x, section_y), 
                           (nav_btn_x + nav_btn_width, section_y + nav_btn_height)):
            gaze_element = "section"
            gaze_index = i
            # Highlighted section button
            draw_rounded_rectangle(menu_interface, (nav_btn_x, section_y), 
                        (nav_btn_x + nav_btn_width, section_y + nav_btn_height), 
                        COLORS['accent'], 6)
            font_color = (255, 255, 255)  # White text
        else:
            # Normal section button
            draw_rounded_rectangle(menu_interface, (nav_btn_x, section_y), 
                        (nav_btn_x + nav_btn_width, section_y + nav_btn_height), 
                        COLORS['panel'], 6)
            # Add subtle border
            cv2.rectangle(menu_interface, (nav_btn_x, section_y), 
                        (nav_btn_x + nav_btn_width, section_y + nav_btn_height), 
                        (60, 60, 60), 1)
            font_color = COLORS['text']
        
        # Section button label with better font
        section_label = f"Section {i+1}"
        text_size = cv2.getTextSize(section_label, font, 0.7, 1)[0]  # Larger font
        text_x = nav_btn_x + int((nav_btn_width - text_size[0]) / 2)
        text_y = section_y + int((nav_btn_height + text_size[1]) / 2)
        cv2.putText(menu_interface, section_label, (text_x, text_y), font, 0.7, font_color, 1)
    
    # Back and Save buttons at the bottom with proper spacing
    btn_width = 120
    btn_height = 50
    btn_gap = 60
    
    # Center the two buttons
    total_btns_width = (btn_width * 2) + btn_gap
    btns_start_x = (menu_interface.shape[1] - total_btns_width) // 2
    btns_y = 590  # Position at the bottom with good spacing
    
    # Back button
    back_btn_x = btns_start_x
    
    # Check if cursor is over back button
    if is_point_in_rect(cursor_pos, (back_btn_x, btns_y), (back_btn_x + btn_width, btns_y + btn_height)):
        gaze_element = "back"
        draw_rounded_rectangle(menu_interface, (back_btn_x, btns_y), 
                    (back_btn_x + btn_width, btns_y + btn_height), 
                    COLORS['accent'], 6)
        font_color = (255, 255, 255)  # White text
    else:
        draw_rounded_rectangle(menu_interface, (back_btn_x, btns_y), 
                    (back_btn_x + btn_width, btns_y + btn_height), 
                    COLORS['panel'], 6)
        # Add subtle border
        cv2.rectangle(menu_interface, (back_btn_x, btns_y), 
                     (back_btn_x + btn_width, btns_y + btn_height), 
                     (60, 60, 60), 1)
        font_color = COLORS['text']
    
    # Back text - improved font
    back_text = "BACK"
    text_size = cv2.getTextSize(back_text, font, 0.9, 2)[0]  # Larger font
    text_x = back_btn_x + int((btn_width - text_size[0]) / 2)
    text_y = btns_y + int((btn_height + text_size[1]) / 2)
    cv2.putText(menu_interface, back_text, (text_x, text_y), font, 0.9, font_color, 2)
    
    # Save button
    save_btn_x = btns_start_x + btn_width + btn_gap
    
    # Check if cursor is over save button
    if is_point_in_rect(cursor_pos, (save_btn_x, btns_y), (save_btn_x + btn_width, btns_y + btn_height)):
        gaze_element = "save"
        draw_rounded_rectangle(menu_interface, (save_btn_x, btns_y), 
                    (save_btn_x + btn_width, btns_y + btn_height), 
                    COLORS['success'], 6)
        font_color = (255, 255, 255)  # White text
    else:
        draw_rounded_rectangle(menu_interface, (save_btn_x, btns_y), 
                    (save_btn_x + btn_width, btns_y + btn_height), 
                    COLORS['panel'], 6)
        # Add subtle border
        cv2.rectangle(menu_interface, (save_btn_x, btns_y), 
                     (save_btn_x + btn_width, btns_y + btn_height), 
                     (60, 60, 60), 1)
        font_color = COLORS['text']
    
    # Save text - improved font
    save_text = "SAVE"
    text_size = cv2.getTextSize(save_text, font, 0.9, 2)[0]  # Larger font
    text_x = save_btn_x + int((btn_width - text_size[0]) / 2)
    text_y = btns_y + int((btn_height + text_size[1]) / 2)
    cv2.putText(menu_interface, save_text, (text_x, text_y), font, 0.9, font_color, 2)
    
    # Draw cursor - add cursor on top of interface
    draw_cursor(menu_interface, cursor_pos[0], cursor_pos[1])

EyeType system starting...
Press ESC to exit or use the Exit button in the interface
Cleaning up resources...


KeyboardInterrupt: 